## Open Street and Phone Number Analysis in Allen, TX 

### Map Area: Allen, TX  

**Motivation** 


The reason why I am choosing Allen in Texas is because I am considering living there. 

In [10]:
# Importing needed libraries
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
import csv
import codecs
import pip

def install(package):
   pip.main(['install', package])

install('cerberus')

import cerberus

def install(package):
   pip.main(['install', package])

install('schema')
# Do the import
import schema as sch


import sqlite3


You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [11]:
# OSM file import
osm_file = "ex_ijn8icTrVTwhvS28Nvi9uBAa3A7aQ.osm"

OSM file contains all the data from Open Map Street.

In [12]:
# re function
# Compile a regular expression pattern into a regular expression object
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

# Expected street types list
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place",
            "Square", "Lane", "Road", "Trail", "Parkway", "Commons", "Alley",
            "Bridge", "Highway", "Circle", "Terrace", "Way"]

We tried to identify here above the regular expressions. We are listing all the possible street types that could be contained in the OSM file. 

In [13]:
# Adding street names in dictionary by type
# Takes 2 arguments: dictionary and string. If string doesn't match pattern adds it to dictionary.
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


In the cell above, we added the street name in the dictionary by type. If there is no match, those names are added to a dictionary. 

In [14]:
# Checking tag for street data content
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


In the cell above, we are checking the street data content from the OSM file. 

In [15]:
def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

In the cell above, we are auditing the file. This will consist of reading the file and parsing it, taking the osm file as input and returning dictionary of different abbreviations of different street types.  


In [16]:
st_types = audit(osm_file)

In the cell above, the audit function we created is being executed. 

In [17]:
# Printing results
pprint.pprint(dict(st_types))

{'110': set(['Greenville Ave #110']),
 '121': set(['SH 121', 'State Hwy 121', 'TX 121']),
 '300': set(['Collin McKinney Pkwy, Suite 300']),
 '510': set(['E Stacy Road Ste 510']),
 'Blvd': set(['Chase Oaks Blvd']),
 'Blvd.': set(['Hardin Blvd.']),
 'Cove': set(['Pine Bluff Cove']),
 'Dr': set(['Lynbridge Dr', 'McDermott Dr', 'Red River Dr', 'Woodson Dr']),
 'Expessway': set(['South Central Expessway']),
 'Expressway': set(['Central Expressway',
                    'North Central Expressway',
                    'S Central Expressway',
                    'South Central Expressway']),
 'Landing': set(['Kemps Landing']),
 'McDermott': set(['McDermott']),
 'Point': set(['Lookout Point']),
 'Rd': set(['Jupiter Rd', 'McDermott Rd']),
 'S': set(['S']),
 'South': set(['Central Expressway South']),
 'St': set(['E Main St']),
 'Tollway': set(['Sam Rayburn Tollway']),
 'Trace': set(['Natchez Trace'])}


In the cell above, we're printing the different street type. There are some streets that are not completely defined. There is a consistency issue here. For example, in the cell above, we have Blvd and Blvd. even though they both mean the same definition. 

Let's create a mapping of abbreviation to their complete names. 

In [18]:
mapping = {"St": "Street",
           "St.": "Street",
           "ST": "Street",
           "Ave": "Avenue",
           "Ave.": "Avenue",
           "Av.": "Avenue",
           "Av": "Avenue",
           "Sq": "Square",
           "CT": "Court",
           "Ct": "Court",
           "DR": "Drive",
           "Dr": "Drive",
           "Dr.": "Drive",
           "Rd.": "Road",
           "Rd": "Road",
           "Pl": "Place",
           "Hwy": "Highway",
           "Ln": "Lane",
           "Blvd": "Boulevard",
           "Blvd.": "Boulevard",
           "Brdg": "Bridge",
           "Ter": "Terrace"
           }

In [19]:
osm_file_2 = "AllenTexas2.osm"

In the cell above, we are creating a resulting file from the mapping. 

 The following function replace abbreviation by full name in string.

In [20]:
def update_name(name, mapping):
    words = name.split(' ')
    last_word = words[-1]
    if last_word in mapping.keys():
        name2 = name.replace(last_word, mapping[last_word])
        return name2
    return name


In [21]:
def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

In the cell above, the function takes OSM file as imput and yields nodes of type from tuple.

In [22]:
def modify_street(old_file, new_file):
    with open(new_file, 'wb') as output:
        output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        output.write('<osm>\n  ')
        for i, element in enumerate(get_element(old_file)):
            for tag in element.iter("tag"):
                if is_street_name(tag):
                    tag.set('v', update_name(tag.attrib['v'], mapping))
            output.write(ET.tostring(element, encoding='utf-8'))
        output.write('</osm>')

In the cell above, the function creates the modification of the origional OSM file by mapping the abbreviation and writing the result in a new file called osm_file_2. 

In [23]:
# Modifying osm file
modify_street(osm_file, osm_file_2)

This function above is being executed. 

In [24]:
# Checking tag for postcode content
zip_type_re = re.compile(r'\d{5}-??')


Cell above is used to check tag for post code contents. 

We are going to sort the zip codes and save them into a dictionary. 

In [25]:
# Sorting zipcodes in different forms and save them to dict
def audit_zip_type(zip_types, zip):
    m = zip_type_re.search(zip)
    if m:
        zip_type = m.group()
        if zip_type not in zip_types:
            zip_types[zip_type].add(zip)
    else:
        zip_types['unknown'].add(zip)

In [26]:
# Checking if tag contains postcode
def is_zip(elem):
    return (elem.attrib['k'] == "addr:postcode")

The above function checks if the tags contain postcodes. 

We are now going to create a function that audits the file based on zipcodes. 

In [27]:
# Auditing file for different variations of same zip
def zip_audit(osmfile):
    osm_file = open(osmfile, "r")
    zip_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zip(tag):
                    audit_zip_type(zip_types, tag.attrib['v'])
    osm_file.close()
    return zip_types

The zip code audit is now being executed below. 

In [28]:
zp_types = zip_audit(osm_file_2)

Let's take a look at the results from the zip code audit.

In [29]:
# Printing results
pprint.pprint(dict(zp_types))

{'75002': set(['75002']),
 '75013': set(['75013']),
 '75023': set(['75023']),
 '75025': set(['75025']),
 '75069': set(['75069']),
 '75070': set(['75070']),
 '75074': set(['75074'])}


For now, for the sample we have above, the zip codes are unique and seem to have reasonable format. 

In [30]:
osm_file_3 = "AllenTexas3.osm"

In the cell above, we're creating a new file container which will get the results from the zip code modification. 

The following function is going to update the wrong zip codes and replace them with the right zip codes. 

In [31]:
def update_zip(zip):
    m = zip_type_re.search(zip)
    if m:
        return m.group()
    else:
        return 'unknown'


In [32]:
def modify_zip(old_file, new_file):
    with open(new_file, 'wb') as output:
        output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        output.write('<osm>\n  ')
        for i, element in enumerate(get_element(old_file)):
            for tag in element.iter("tag"):
                if is_zip(tag):
                    tag.set('v', update_zip(tag.attrib['v']))
            output.write(ET.tostring(element, encoding='utf-8'))
        output.write('</osm>')

We are now going to execute the zip code modification and write the results in osm_file_3. 

In [33]:
modify_zip(osm_file_2, osm_file_3)

Now we're going to audit the phone information.

In [34]:
# Checking the tag for phone content
phone_type_re = re.compile(r'\d{3}\)?-?\s?.?\d{3}\s?-?\s?.?\d{4}')


In the code above, we're checking the type for phone contents. Let's compile any phone number in bad format. 

In [35]:
phone_re = re.compile('\.|\)|\s|-')

We want our phone numbers to start with 1 and have an area code. We also want our phone numbers to have 10 digits in total ranging from 0 to 9 to be valid. The function below addresses this. 

In [36]:
def audit_phone_type(phone_types, phone):
    m = phone_type_re.search(phone)
    if m:
        phone_type = m.group()
        if phone_type not in phone_types:
            new_phone = phone_re.sub('', phone_type)
            new_phone = ('+1-' + new_phone[:3] + '-' +
                         new_phone[3:6] + '-' + new_phone[6:])
            phone_types[new_phone].add(phone)
    else:
        phone_types['unknown'].add(phone)

Now, let us check if the tag contains "phone."

In [37]:
# Checking if tag contains phone
def is_phone(elem):
    return (elem.attrib['k'] == "phone")

Let us now audit the phone information.

In [38]:
def phone_audit(osmfile):
    osm_file = open(osmfile, "r")
    phone_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_phone(tag):
                    audit_phone_type(phone_types, tag.attrib['v'])
    osm_file.close()
    return phone_types

Let's now execute the phone audit function created above. 

In [41]:
ph_types = phone_audit(osm_file_3)

Let us now print the results of the phone audit. 

In [42]:
pprint.pprint(dict(ph_types))

{'+1-214-383-3500': set(['214-383-3500']),
 '+1-214-383-4008': set(['+1214-383-4008']),
 '+1-214-383-5353': set(['(214) 383-5353']),
 '+1-214-383-9712': set(['+1 214 383 9712']),
 '+1-214-509-4653': set(['+1 214 509 4653']),
 '+1-214-644-0280': set(['(214) 644-0280']),
 '+1-214-667-2100': set(['+1 214 667 2100']),
 '+1-469-752-0600': set(['+1 (469) 752-0600']),
 '+1-469-854-6681': set(['469-854-6681']),
 '+1-972-363-2101': set(['+1 972 363 2101']),
 '+1-972-390-1040': set(['+1 972 390 1040']),
 '+1-972-390-9917': set(['(972) 390-9917']),
 '+1-972-578-9779': set(['+1-972-578-9779']),
 '+1-972-649-6913': set(['(972) 649-6913']),
 '+1-972-678-4700': set(['972-678-4700']),
 '+1-972-678-7000': set(['+1 972-678-7000']),
 '+1-972-727-0250': set(['+1 972 727 0250']),
 '+1-972-912-1097': set(['+1 972 912 1097'])}


We see that phone numbers do not start with 1 that identifies US country code. Other phone numbers do not separate the digit 1 from the other numbers. Other numbers are with or without dashes. There is a problem of consistency here. 

Let us update the format of these phone numbers contained in the OSM file. 

Before that, let's a create a new data container called OSM_file_4.

In [43]:
osm_file_4 = "AllenTexas4.osm"

In [44]:
pprint.pprint(dict(ph_types))

{'+1-214-383-3500': set(['214-383-3500']),
 '+1-214-383-4008': set(['+1214-383-4008']),
 '+1-214-383-5353': set(['(214) 383-5353']),
 '+1-214-383-9712': set(['+1 214 383 9712']),
 '+1-214-509-4653': set(['+1 214 509 4653']),
 '+1-214-644-0280': set(['(214) 644-0280']),
 '+1-214-667-2100': set(['+1 214 667 2100']),
 '+1-469-752-0600': set(['+1 (469) 752-0600']),
 '+1-469-854-6681': set(['469-854-6681']),
 '+1-972-363-2101': set(['+1 972 363 2101']),
 '+1-972-390-1040': set(['+1 972 390 1040']),
 '+1-972-390-9917': set(['(972) 390-9917']),
 '+1-972-578-9779': set(['+1-972-578-9779']),
 '+1-972-649-6913': set(['(972) 649-6913']),
 '+1-972-678-4700': set(['972-678-4700']),
 '+1-972-678-7000': set(['+1 972-678-7000']),
 '+1-972-727-0250': set(['+1 972 727 0250']),
 '+1-972-912-1097': set(['+1 972 912 1097'])}


In [45]:
# This function updates format of phone numbers to right one.
def update_phone(phone):
    m = phone_type_re.search(phone)
    if m:
        new_phone = phone_re.sub('', m.group())
        return ('+1-' + new_phone[:3] + '-' + new_phone[3:6] +
                '-' + new_phone[6:])
    else:
        return phone

In [46]:
# This function replace wrong phone formats in osm file
def modify_phone(old_file, new_file):
    with open(new_file, 'wb') as output:
        output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        output.write('<osm>\n  ')
        for i, element in enumerate(get_element(old_file)):
            for tag in element.iter("tag"):
                if is_phone(tag):
                    tag.set('v', update_phone(tag.attrib['v']))
            output.write(ET.tostring(element, encoding='utf-8'))
        output.write('</osm>')

In [47]:
# The following will update the wrong phone information with the right one
modify_phone(osm_file_3, osm_file_4)

It is now time to create the csv files results and embbed them into sql format. We will then use sqlite3 to embbed those files in a database.

In [48]:
# Path to new csv files
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

In [49]:
# Regular expression compilers
LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCH = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

In [50]:
# Note: The schema is stored in a .py file in order to take advantage of the
# int() and float() type coercion functions. Otherwise it could easily stored as
#as JSON or another serialized format.

schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}


In [51]:
# Importing schema for pransformation from schema.py file
SCHEMA = sch.schema

In [52]:
# Fields of new csv files
#The schema describes what fields will get into the database and their format

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version',
               'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']



In [53]:
# Main function for transformation of XML data to Python dict
def shape_element(element, node_attr_fields=NODE_FIELDS,
                  way_attr_fields=WAY_FIELDS, prob_ch=PROBLEMCH,
                  default_tag_type='regular'):
    tag_attribs = {}
    way_nodes = []
    tags = []
    count = 0
    if element.tag == 'node':
        tagfields = node_attr_fields
    elif element.tag == 'way':
        tagfields = way_attr_fields

    if element.tag == 'node' or 'way':
        for attrib in element.attrib:
            if attrib in tagfields:
                tag_attribs[attrib] = element.attrib[attrib]
    for subelem in element:
        if subelem.tag == 'tag' and prob_ch.match(subelem.attrib['k']) == None:
            tag = {}
            tag['id'] = tag_attribs['id']
            tag['value'] = subelem.attrib['v']
            key = subelem.attrib['k']
            tag['key'] = key[key.find(':') + 1:]
            if ':' in key:
                tag['type'] = key[:key.find(':')]
            else:
                tag['type'] = default_tag_type
            tags.append(tag)
        elif subelem.tag == 'nd':
            way_node = {}
            way_node['id'] = tag_attribs['id']
            way_node['node_id'] = subelem.attrib['ref']
            way_node['position'] = count
            count += 1
            way_nodes.append(way_node)

    if element.tag == 'node':
        return {'node': tag_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': tag_attribs, 'way_nodes': way_nodes, 'way_tags': tags}



In [54]:
# We're validating element to match schema. This function validate every element to match the schema.

def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )




In [55]:
# Extend csv.DictWriter to handle Unicode input

class UnicodeDictWriter(csv.DictWriter, object):
    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
                                                    k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in
                                                    row.iteritems()
                                                    })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)



In [56]:
# Main function processing osm file to 5 csv files. This function takes the osm file result and produces
# the csv files as final results
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
            codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
            codecs.open(WAYS_PATH, 'w') as ways_file, \
            codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
            codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])




In [57]:
#let us now process the osm file and get the results
process_map(osm_file_4, validate=True)

The data was successfully processed and the output file was generated. The output file is included in the submission and it contains the filtered and corrected map data.

The size of the jupyter notebook file for this project is 29 KB. 
There are 116849 of nodes in our dataset and there are 15424 ways. There are 444067616996  nodes in the biggest way in this database.

#The above results were generated by sql queries which are provided below: 

The queries we used are below: 

BUILDING SQL DB BY SCHEMA  

In this section we will build empty SQL database and create table schema.
We'll use sqlite3 shell for this purpose. It's needed to create DB and read create_db.csv file. Next command will execute create_csv.db file by executing next command:

.read create_csv.db

This file contains next rows:

CREATE TABLE nodes ( id INTEGER PRIMARY KEY NOT NULL, lat REAL, lon REAL, user TEXT, uid INTEGER, version INTEGER, changeset INTEGER, timestamp TEXT );
CREATE TABLE nodes_tags ( id INTEGER, key TEXT, value TEXT, type TEXT, FOREIGN KEY (id) REFERENCES nodes(id) );
CREATE TABLE ways ( id INTEGER PRIMARY KEY NOT NULL, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp TEXT );
CREATE TABLE ways_tags ( id INTEGER NOT NULL, key TEXT NOT NULL, value TEXT NOT NULL, type TEXT, FOREIGN KEY (id) REFERENCES ways(id) );
CREATE TABLE ways_nodes ( id INTEGER NOT NULL, node_id INTEGER NOT NULL, position INTEGER NOT NULL, FOREIGN KEY (id) REFERENCES ways(id), FOREIGN KEY (node_id) REFERENCES nodes(id) );
   .mode csv
   .import nodes.csv nodes
   .import ways.csv ways
   .import nodes_tags.csv nodes_tags
    delete from nodes_tags where id = 'id';
    .import ways_tags.csv ways_tags
    delete from ways_tags where id = 'id';
    .import ways_nodes.csv ways_nodes
   delete from ways_nodes where id = 'id';


SQLite3 QUERING 

Establishing connection and cursor
conn = sqlite3.connect("osm.db")
cursor = conn.cursor()

Executing and printing
cursor.execute("select count(id) from nodes;")
print 'There are {} nodes in database.'.format(cursor.fetchall()[0][0])

cursor.execute("select count(id) from ways;")
print 'There are {} ways in database.'.format(cursor.fetchall()[0][0])

cursor.execute("select count(distinct(uid)) from (select uid from nodes union select uid from ways);")
print 'There are {} uniqe users in database.'.format(cursor.fetchall()[0][0])

cursor.execute("select id, count(*) as nodes_count from ways_nodes group by id order by nodes_count desc limit 1;")
way_id, count = cursor.fetchall()[0]
print "There're {} nodes in the biggest way in database. Way id is {}.".format(count, way_id)

cursor.execute("select * from ways_tags where id = {};".format(way_id))
print 'This way is:'
pprint.pprint(cursor.fetchall())

cursor.execute("select count(key) from ways_tags where key = 'bridge' and value != 'yes' group by key;")
print "There are {} bridges in Pitt. That's a second Venice.".format(cursor.fetchall()[0][0])

cursor.execute("select value, count(*) as count from nodes_tags where key = 'postcode' group by value order by count desc limit 5;")
pprint.pprint(cursor.fetchall())


sqlite3 nodes.db
CREATE TABLE nodes ( id INTEGER PRIMARY KEY NOT NULL, lat REAL, lon REAL, user TEXT, uid INTEGER, version INTEGER, changeset INTEGER, timestamp TEXT );

One suggestion for improving this analysis is to run the queries from within the jupyter environment because this would shorten the number of steps required to count the nodes and ways. 
We're setting up a database and doing some queries from within Jupyter here: One benefit in doing these following steps below would be that it allow us to count the nodes faster and more efficiently. One possible problem is that the syntax may differ for some of the queries (excluding the one below). 


conn = sqlite3.connect('example.db')
c = conn.cursor()
c.execute('''CREATE TABLE nodes ( id INTEGER PRIMARY KEY NOT NULL, lat REAL, lon REAL, user TEXT, uid INTEGER, version INTEGER, changeset INTEGER, timestamp TEXT );''')
c.execute('''CREATE TABLE nodes_tags ( id INTEGER, key TEXT, value TEXT, type TEXT, FOREIGN KEY (id) REFERENCES nodes(id) );''')
c.execute('''CREATE TABLE ways ( id INTEGER PRIMARY KEY NOT NULL, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp TEXT );''')
c.execute('''CREATE TABLE ways_tags ( id INTEGER NOT NULL, key TEXT NOT NULL, value TEXT NOT NULL, type TEXT, FOREIGN KEY (id) REFERENCES ways(id) );''')
c.execute('''CREATE TABLE ways_nodes ( id INTEGER NOT NULL, node_id INTEGER NOT NULL, position INTEGER NOT NULL, FOREIGN KEY (id) REFERENCES ways(id), FOREIGN KEY (node_id) REFERENCES nodes(id) );''')
c.execute('''mode csv''')
c.execute('''import nodes.csv nodes''')
c.execute('''import ways.csv ways''')
c.execute('''import nodes_tags.csv nodes_tags''')
c.execute('''delete from nodes_tags where id = 'id';''')
c.execute('''import ways_tags.csv ways_tags''')
c.execute('''delete from ways_tags where id = 'id';''')
c.execute('''import ways_nodes.csv ways_nodes''')
c.execute('''delete from ways_nodes where id = 'id';''')

One possible way in improving our data set in the future may be that in this analysis, it was discovered that in many instances, street names were abbreviated in multiple ways and the abbreviations had to be processed and filtered. For example, drive was originally inconsistently named such as DR, Dr, and Dr. Thus, we needed to filter those three terms as Drive in order for the code to run. 

Another issue we had was that there were 22 phone numbers in which 18 of them were inconsistent (i.e. there were extra plus signs, minus signs, inconsistent spacing, etc.) and we had to clean up these 18 numbers in the right format. In the future, in order to improve our data to be more consistent, we can automatically format the user input as soon as they are entered and before they are saved in such a way that they cannot enter the phone numbers in a wrong format. 

